<a href="https://colab.research.google.com/github/flanaganc04/My-own-Neural-Net/blob/main/Digit_Recognizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
# Import packages
from google.colab import drive
import os
import datetime
# import matplotlib as mpl
# import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import math

In [43]:
# Mount Google Drive and try not to get yelled at
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
def hiddenLayer(neurons: int, matrix: np.ndarray) -> np.ndarray:
    """
    Builds a random weight matrix, multiplies it by the transpose
    of the input matrix, and returns the resulting hidden layer matrix and weight matrix for later in backpropagation.

    Parameters
    ----------
    neurons : int
        Number of neurons in the layer.
    matrix : np.ndarray
        Input data (e.g., training or testing data).

    Returns
    -------
    np.ndarray
        The computed hidden layer matrix and the weight matrix.
    """

    # Create a random weight matrix with values uniformly distributed between -1 and 1
    weightMatrix = np.random.uniform(low=-1.0, high=1.0, size=(neurons, matrix.shape[1]))
    # Compute the hidden layer activations by performing matrix multiplication
    hiddenMatrix = matrix @ weightMatrix.T
    # Return both the hidden layer activations and the weight matrix
    return hiddenMatrix, weightMatrix



In [45]:
def softMax(matrix: np.ndarray) -> np.ndarray:
  """
    Takes a matrix, and applies the soft max function to each element. x_i,j = e^(x_i,j)/sum(e^(x_i,j)). So take the exponential of the rows, sum it, and divide each element by that sum.

    Parameters
    ----------
    matrix : np.ndarray
        Array in which soft max should be applied (e.g., hidden layer or output layer).

    Returns
    -------
    np.ndarray
        The computed soft max matrix.
    """
  for y in range(matrix.shape[0]):
    sum = 0
    # Compute the denominator: sum of exponentials for each row
    try:
      for x in range(matrix.shape[1]):
        sum += math.exp(matrix[y][x])
    except OverflowError:
      print(f'Sum e^(x_i) OverflowError: row:{y} and column:{x}')
    except ZeroDivisionError:
      print(f'Sum e^(x_i) OverflowError: row:{y} and column:{x}')
    # Apply softmax normalization for each element in the row
    try:
      for x in range(matrix.shape[1]):
        matrix[y][x] = math.exp(matrix[y][x]) / sum
    except OverflowError:
      print(f'[e^(x_i)/sum] OverflowError: row:{y} and column:{x}')
    except ZeroDivisionError:
      print(f'[e^(x_i)/sum] OverflowError: row:{y} and column:{x}')
  # Return the softmax-normalized matrix
  return matrix

In [57]:
def OneHotEncoder(matrix: np.ndarray) -> np.ndarray:
    """
    Convert an integer label matrix (values 0..N) into a one-hot encoded matrix.

    Parameters
    ----------
    matrix : np.ndarray
        Input array of integer class labels (shape: (n_samples,) or (n_samples, 1)).

    Returns
    -------
    one_hot_matrix : np.ndarray
        One-hot encoded array of shape (n_samples, num_classes).
    """
    # Ensure integer dtype (in case it's float or object from pandas)
    matrix = matrix.astype(int).flatten()
    # Determine number of classes (assumes classes are labeled 0..N)
    num_classes = np.max(matrix) + 1
    # Create one-hot encoding
    one_hot_matrix = np.eye(num_classes)[matrix]
    return one_hot_matrix


In [47]:
# Import data
test_data = pd.read_csv('/content/drive/MyDrive/Digit Recognizer/mnist_test.csv')
train_data = pd.read_csv('/content/drive/MyDrive/Digit Recognizer/mnist_train.csv')

X_train = train_data.iloc[0:, 1:]  # Features
X_train_np = X_train.to_numpy() *(1/255)#
# print(X_train_np[0])

Y_train = train_data.iloc[0:, 0:1]   # Target variable
Y_train_np = Y_train.to_numpy()
# print(Y_train)

X_test = test_data.iloc[1:, 1:]  # Features
X_test_np = X_test.to_numpy() *(1/255)
# print(X_test)

Y_test = test_data.iloc[1:, 0:1]   # Target variable
Y_test_np = Y_test.to_numpy()
# print(Y_test)

In [48]:
# Make layer 1
layer1, weights1 = hiddenLayer(25, X_train_np)
softLayer1 = softMax(layer1)


In [49]:
# Make layer 2
layer2, weights2 = hiddenLayer(10, softLayer1)
softLayer2 = softMax(layer2)


In [50]:
print(layer2[5])

[0.08331298 0.13982028 0.05443507 0.14321315 0.0622096  0.18202432
 0.03333699 0.076241   0.15414874 0.07125788]


In [51]:
biasMatrix = np.ones((1,10))
biasMatrix = np.array(biasMatrix)
outputMatrix = softLayer2 + biasMatrix
print(outputMatrix[0])

[1.159936   1.10952711 1.08795249 1.03532658 1.16024736 1.1166047
 1.05313956 1.03179493 1.0903607  1.15511056]


In [52]:
softOput = softMax(outputMatrix)
print(softOput[0])

[0.10606087 0.10084698 0.09869454 0.09363495 0.1060939  0.10156326
 0.09531782 0.09330485 0.09893251 0.10555031]


In [53]:
indices = np.argmax(softOput, 1)

In [54]:
# print(indices)

In [55]:
Y_train_onehot = OneHotEncoder(Y_train_np)
print(Y_train_onehot.shape)
print(Y_train_onehot)

(60000, 10)
[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]
